## Imports

In [1]:
#  https://www.kaggle.com/scratchpad/notebookf57ae0f829/edit

In [2]:
!pip install git+https://github.com/APCLab/jieba-tw.git

  Cloning https://github.com/APCLab/jieba-tw.git to /tmp/pip-req-build-ngfhgu2j
  Running command git clone -q https://github.com/APCLab/jieba-tw.git /tmp/pip-req-build-ngfhgu2j
  Created wheel for jieba: filename=jieba-0.39.1-cp37-none-any.whl size=6603876 sha256=9f0119dd7d12ce07fa08fcf3ac5d66f76f40bb1407833982fd961fea89255f87
  Stored in directory: /tmp/pip-ephem-wheel-cache-041c7nia/wheels/9d/80/40/e3449d06ed4c0a5f96091452072765e804e5df8d353192a6b3
Successfully built jieba
  Found existing installation: jieba 0.42.1
    Uninstalling jieba-0.42.1:
      Successfully uninstalled jieba-0.42.1


In [3]:
import pandas as pd
import numpy as np
import csv
import os
import jieba


## Reading in the Dataframe

In [4]:
from google.colab import drive
drive.mount("/content/drive")
path =  "/content/drive/Shareddrives/AI_project/"

Mounted at /content/drive


In [8]:
df = pd.read_csv(path + 'data/news_all.csv')
df_3col = df[['media', 'category', 'title']]


In [9]:
sector = df_3col.groupby('media')
sector.size()

media
chinatimes     9129
cna            4060
cts            1052
ebc            1414
ettoday        8469
ltn            8518
media             1
setn           5452
storm          1281
udn           11279
dtype: int64

In [ ]:
# !!!!!Modify the media here
media = "聯合"
m = sector.get_group(media)

In [ ]:
mc = m.groupby('category')
mc.size()

category
Oops      803
全球       6583
兩岸       2928
地方       7943
數位        841
文教       2052
生活       6455
產經       8258
社會       6134
股市       2738
要聞      13394
評論        205
運動       4045
閱讀        117
dtype: int64

#Snowlp

In [ ]:
!pip install snownlp
from snownlp import SnowNLP

In [ ]:
# estimation for title
df = pd.read_csv(path + 'data/1_column_title/simple_label_2kind.csv')
test_title = df['title']
test_label = df['label']


i = 0
correct_cnt = 0.0
for line in test_title:
  s = SnowNLP(line)
  grade = s.sentiments
  label = 0 if grade<0.5 else 1
  if label != test_label[idx]:
    print(line, test_label[i], " wrong label : ", label)
  else : 
    correct_cnt += 1
  i += 1
print("正確率 : ", correct_cnt*100/len(test_label),"%")

灭绝40年巨獭现身了！保育员吓坏录影：没人会相信我 1  wrong label :  0
疫情期间穷到断炊？炒饭店老板霸气助人：拢来我请 1  wrong label :  0
打疫苗惊见80年前国小同学！2阿公互呛：明天要去把七辣 1  wrong label :  0
疫情停工薪水照发！这些明星老板超佛心 1  wrong label :  0
75岁老母亲被取消疫苗！　「超暖1句话」喷洋葱…蓝心湄：我没有用 1  wrong label :  0
在台日人不打新冠疫苗 台大医曝原因 1.1万网狂讚 1  wrong label :  0
黄珊珊》台湾致死率全世界第一 太多人枉死 0  wrong label :  0
雪碧示范5套火辣泳装「没奶不要穿」 这一款下水变透明 1  wrong label :  0
日本公司推「胖子出租」！聊天、搬家、陪吃饭都OK 1  wrong label :  0
正妹外送员半夜接医院订单 霸气「all in现金」买咖啡送暖 1  wrong label :  0
友达9年前下定决心解决这问题 94％废水回收率胜台积电 1  wrong label :  0
贡寮阿嬷打疫苗隔天还采笋 总统视讯关心相约包粽 1  wrong label :  0
猝死案例频传 高雄长辈接种率急降 0  wrong label :  0
土耳其里拉周线跌幅 恐创2个月最大 0  wrong label :  0
邻长若染疫如何服务里民？地方吁中央重视 0  wrong label :  0
公开3＋11会议记录遭挡 陈思宇痛批：立委现形记 0  wrong label :  0
失业宅在家2个月还确诊！她叹：谁能比我惨 0  wrong label :  0
祕鲁矿工巴士坠山谷 27死12人伤 0  wrong label :  0
黄暐瀚2天前问「若买BNT的是张忠谋」 网惊：铁口神算 1  wrong label :  0
男友长太丑！妈妈见完面气炸 狠甩她1巴掌 0  wrong label :  0
被控违反国安法  壹传媒和苹果日报两名高层应讯 0  wrong label :  0
「红色丰田86」台7乙线逆向狂飙 三峡拓海哥雷残骨折 0  wrong label :  0
天降木板！少女被爆头浑身血惨死 朋友目睹崩溃尖叫 0  w

In [ ]:
# estimation for content
'''
import re
pos = []
nu = []
neg = [] 
pos_cnt = 0
nu_cnt = 0
neg_cnt = 0
df = pd.read_csv(path + 'data/1_column_title/label_all.csv')

data = mc.get_group('社會')['content']
for line in data:
  simple = SnowNLP(line).han #轉簡體
  r = re.findall(r'[^，、。,;?!\s]+', simple)
  grade = 0.0
  cnt_sentence =0
  for sentence in r:
    s = SnowNLP(sentence)
    grade += s.sentiments
    cnt_sentence +=1
  mean_grade = grade/cnt_sentence
  if (mean_grade)<0.3 : 
    neg_cnt+=1
    neg.append(simple)
  else:
    if mean_grade>0.6:
      pos_cnt+=1
      pos.append(simple)
    else :
      nu_cnt+=1
      nu.append(simple)
'''

# Testing

In [ ]:
df_test = pd.
for line in data:
  simple = SnowNLP(line).han #轉簡體
  r = re.findall(r'[^，、。,;?!\s]+', simple)
  grade = 0.0
  cnt_sentence =0
  for sentence in r:
    s = SnowNLP(sentence)
    grade += s.sentiments
    cnt_sentence +=1
  mean_grade = grade/cnt_sentence
  if (mean_grade)<0.3 : 
    neg_cnt+=1
    neg.append(simple)
  else:
    if mean_grade>0.6:
      pos_cnt+=1
      pos.append(simple)
    else :
      nu_cnt+=1
      nu.append(simple)

# Output

In [ ]:
path =  "/content/drive/Shareddrives/AI_project/"
filepath = path+'result/'+ media +'/'+media
print(pos_cnt, nu_cnt, neg_cnt)

path = filepath+'.txt'
f = open(path, 'w')
f.write(str(pos_cnt)+" ")
f.close()
f = open(path, 'a')
f.write(str(nu_cnt)+" ")
f.write(str(neg_cnt))
f.close()

206 5556 372


In [ ]:
print(pos[0])
print(nu[0])
print(neg[0])

['本月8日发生一起屏东曾女受到黄男掳人杀害的案件，尽管曾女曾经二度向警方报案，要警察以性骚扰处理，却仍传出憾事，内政部长徐国勇上午在内政委员会接受质询时表示，已经跟行政院政务委员罗秉成联系，将在本会期将「跟踪骚扰法」行政院版本送立法院审议。法务部检察司的主任检察官连思藩也表示，目前没有相关法律，法律通过后会配合办理。屏东发生一起曾女受到黄男掳人杀害的案件，尽管曾女曾经二度向警方报案，要警察以性骚扰处理，却仍传出憾事，内政部长徐国勇（右）及法务部检察司的主任检察官连思藩等上午在内政委员会接受质询，徐国勇表示将在本会期将「跟踪骚扰法」行政院版本送立法院审议。记者郑超文摄影']
['高雄市三民区一名陈姓男子昨天晚间10时许骑机车出门，在路口又回转折返，险与骑红牌重机的锺姓男子擦撞，他认为锺男瞪他一眼，追上去呛声，并打电话叫儿子到场助阵，陈男儿子疑持西瓜刀与锺男争执，附近民众听到声音目睹报警，警方到场未见刀械，因双方互控恐吓，全带回调查。', '据了解，陈男（58岁）昨天骑车出门突然想起有东西没拿，到教仁路口回转时，与从教仁路右转骑红牌重机的锺男（32岁）差点发生擦撞，锺男看了陈男一眼后继续往前骑，陈男却加速往前与锺男发生争执，两人越吵越凶，之后在大顺三路口停下。', '陈男打电话叫儿子到场，陈男儿子以为父亲发生什么事，赶到大顺路口时，看到锺男与父亲推挤，冲过去呛「你干嘛推我爸！」锺男则觉得陈男莫名其妙骑车堵他，还找人恐吓，附近民众听到吵闹声连忙报警。', '警方到场时现场3人已没有吵架情形，也没看到刀械，但双方坚持提告恐吓，锺还控陈男儿子持刀，警方全带回，询后函送高雄地检署侦办。高雄市三民区一名陈姓男子昨天晚间10时许骑机车出门，疑似认为骑行红牌重机的锺姓男子转头瞪他，两人发生争执。记者陈玫伶／翻摄高雄市三民区一名陈姓男子昨天晚间10时许骑机车出门，疑似认为骑行红牌重机的锺姓男子转头瞪他，两人发生争执。记者陈玫伶／翻摄高雄市三民区一名陈姓男子昨天晚间10时许骑机车出门，疑似认为骑行红牌重机的锺姓男子转头瞪他，两人发生争执。记者陈玫伶／翻摄']
['全国疫情警戒提升到第三级，彰化县警察局溪湖分局接获民众检举，辖区内仍有娱乐场所拉下铁门「假防疫，真营业」，昨天晚上突击，果然查获某按摩馆有4名客人在包厢饮酒作乐，今天警方依违反防疫相关规定函送裁处。', '溪湖地区列管63家应

In [ ]:
p = pd.DataFrame(pos)
p.to_csv(filepath+"pos.csv")

nu = pd.DataFrame(nu)
nu.to_csv(filepath+"nu.csv")

neg = pd.DataFrame(neg)
neg.to_csv(filepath+"neg.csv")